# RAG 시스템 - 뷰티 코칭 AI (Hybrid: HuggingFace + OpenAI)

이 노트북은 PDF 문서를 기반으로 한 RAG(Retrieval-Augmented Generation) 시스템입니다.

## 하이브리드 접근 (비용 효율적!) ⭐
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어 지원)
- **LLM**: OpenAI GPT-4o-mini (유료, 고품질, 비전 지원)

## 주요 기능
- PDF 문서 로드 및 전처리
- HuggingFace 다국어 임베딩 및 Chroma DB 저장 (무료!)
- 한글 쿼리로 영어 논문도 검색 가능
- OpenAI GPT-4o-mini 비전 모델을 사용한 이미지+텍스트 질의응답

## 1. 라이브러리 Import

In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import re
import json
import time
import os
import base64
from pathlib import Path

load_dotenv()

# OpenAI API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY가 설정되지 않았습니다. .env 파일을 확인하세요.")
else:
    print("✅ OpenAI API 키가 설정되었습니다.")

✅ OpenAI API 키가 설정되었습니다.


## 2. 유틸리티 함수 정의

In [2]:
def clean_text(text):
    """텍스트 공백 정리 (PDF OCR 오류 대응 강화)"""
    
    # 1. 단어 내부의 과도한 공백 제거 (가장 공격적)
    # "T r a d i t i o n a l" -> "Traditional"
    # 패턴: 1-2글자 + 공백이 3번 이상 반복
    text = re.sub(r'\b(\w{1,2})\s+(?=\w{1,2}\s+\w)', r'\1', text)
    
    # 2. 남은 단일 문자 사이 공백 제거
    # "t h e" -> "the"
    text = re.sub(r'\b(\w)\s+(\w)\s+(\w)\b', r'\1\2\3', text)
    text = re.sub(r'\b(\w)\s+(\w)\b', r'\1\2', text)
    
    # 3. 특수문자와 단어 사이 불필요한 공백 제거
    # "ﬂ a t" -> "flat"
    text = re.sub(r'(\S)\s+(\S)(?=\s+\S{1,2}\s+|\s+\S{1,2}[,.])', r'\1\2', text)
    
    # 4. 여러 개의 연속된 공백을 하나로
    text = re.sub(r' {2,}', ' ', text)
    
    # 5. 탭을 공백으로
    text = text.replace('\t', ' ')
    
    # 6. 줄바꿈 전후 하이픈 처리 (단어 분리)
    # "dis-\nrupting" -> "disrupting"
    text = re.sub(r'-\s*\n\s*', '', text)
    
    # 7. 3개 이상의 연속된 줄바꿈을 2개로
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # 8. 각 줄의 앞뒤 공백 제거
    lines = [line.strip() for line in text.split('\n')]
    
    # 9. 빈 줄과 너무 짧은 줄 제거 (노이즈 제거)
    lines = [line for line in lines if len(line) > 1]
    
    text = '\n'.join(lines)
    
    return text.strip()

def load_single_pdf(pdf_path):
    """단일 PDF 파일 로드 (PyMuPDFLoader 사용)"""
    try:
        loader = PyMuPDFLoader(str(pdf_path))
        docs = loader.load()
        return docs, "PyMuPDFLoader"
    except Exception as e:
        # 에러 정보를 함께 반환
        raise Exception(f"PyMuPDFLoader 실패: {str(e)}")

def load_documents():
    """PDF 문서 로드"""
    
    # data 폴더의 모든 PDF 파일 찾기
    pdf_files = list(Path("data/").rglob("*.pdf"))
    print(f"📁 발견된 PDF 파일: {len(pdf_files)}개\n")
    
    all_documents = []
    failed_files = []
    loader_stats = {}
    
    for pdf_path in pdf_files:
        try:
            print(f"  로딩: {pdf_path.name}...", end=" ")
            docs, loader_name = load_single_pdf(pdf_path)
            all_documents.extend(docs)
            loader_stats[loader_name] = loader_stats.get(loader_name, 0) + 1
            print(f"✅ ({len(docs)}페이지, {loader_name})")
        except Exception as e:
            print(f"❌ 실패: {str(e)}")
            failed_files.append((pdf_path.name, str(e)))
    
    print(f"\n✅ 총 {len(all_documents)}개 페이지 로드 완료")
    print(f"📊 사용된 로더: {loader_stats}")
    
    if failed_files:
        print(f"\n⚠️ {len(failed_files)}개 파일 로드 실패:")
        for filename, error in failed_files:
            print(f"  - {filename}: {error}")
    
    # 텍스트 정리 적용
    print("\n🧹 텍스트 정리 중...")
    for i, doc in enumerate(all_documents):
        original_length = len(doc.page_content)
        doc.page_content = clean_text(doc.page_content)
        cleaned_length = len(doc.page_content)
        
        # 정리 효과 확인 (첫 페이지만)
        if i == 0 and original_length > 0:
            print(f"\n📊 첫 페이지 정리 효과:")
            print(f"  원본 길이: {original_length}자")
            print(f"  정리 후: {cleaned_length}자")
            print(f"  감소율: {(1 - cleaned_length/original_length)*100:.1f}%")
            print(f"\n첫 200자 미리보기:")
            print(doc.page_content[:200])
            print("...")
    
    return all_documents

def chunk_doc(documents, chunk_size=1000, chunk_overlap=200):
    """문서를 청크로 분할 - 개선된 설정
    
    Args:
        chunk_size: 청크 크기 (기본 1000자 - 더 큰 의미 단위 유지)
        chunk_overlap: 청크 간 중복 (기본 200자 - 문맥 연속성 유지)
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        # separators 순서 개선: 큰 의미 단위부터 분할
        separators=[
            "\n\n\n",  # 여러 단락 구분
            "\n\n",    # 단락 구분
            "\n",      # 줄바꿈
            "。",      # 한글 마침표
            ". ",      # 영문 문장 끝
            "! ",      # 느낌표
            "? ",      # 물음표
            "；",      # 세미콜론
            "，",      # 쉼표
            " ",       # 공백 (최후의 수단)
            ""         # 문자 단위 (최후의 최후)
        ],
        keep_separator=True,  # 구분자 유지
        length_function=len,
    )
    splits = text_splitter.split_documents(documents)
    
    # 청크 품질 확인
    print(f"\n📊 청크 통계:")
    print(f"  총 청크 수: {len(splits)}")
    if len(splits) > 0:
        avg_length = sum(len(s.page_content) for s in splits) / len(splits)
        min_length = min(len(s.page_content) for s in splits)
        max_length = max(len(s.page_content) for s in splits)
        print(f"  평균 청크 길이: {avg_length:.0f}자")
        print(f"  최소/최대: {min_length}/{max_length}자")
        print(f"\n첫 번째 청크 미리보기 (500자):")
        print(splits[0].page_content[:500])
        print("...")
    
    return splits

def save_documents(documents, output_path):
    """문서를 JSONL 형식으로 저장"""
    with open(output_path, 'w', encoding='utf-8') as f:
        for i, page in enumerate(documents):
            data = {
                'page': i + 1,
                'text': page.page_content,
                'metadata': {
                    'source': page.metadata.get('source', 'unknown'),
                    'page': page.metadata.get('page', i)
                }
            }
            f.write(json.dumps(data, ensure_ascii=False) + '\n')

def encode_image(image_path):
    """이미지를 base64로 인코딩"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

print("✅ 유틸리티 함수 정의 완료 (개선된 청킹 설정)")

✅ 유틸리티 함수 정의 완료 (개선된 청킹 설정)


## 3. 문서 로드 및 전처리

**이 셀은 한 번만 실행하세요!**

In [3]:
print("문서 로드 중...")
documents = load_documents()
print(f"✅ 총 {len(documents)}개의 문서 로드 완료")

print("\n문서 청킹 중...")
chunked_documents = chunk_doc(documents)
print(f"✅ 총 {len(chunked_documents)}개의 청크 생성 완료")

print("\n문서 저장 중...")
save_documents(chunked_documents, 'tmp.jsonl')
print("✅ tmp.jsonl 저장 완료")

문서 로드 중...
📁 발견된 PDF 파일: 171개

  로딩: 헤어스타일 변화로 인한 감정 치유와 회복 탄력성의 관계에서 자존감의 매개 효과.pdf... ✅ (14페이지, PyMuPDFLoader)
  로딩: 안면 노화 징후 완화를 위한 구강안면 근기능 개입의 효과-PubMed_Central.pdf... ✅ (24페이지, PyMuPDFLoader)
  로딩: 외모·자아수용·퍼펙셔니즘.pdf... ✅ (8페이지, PyMuPDFLoader)
  로딩: 젊음을 위한 얼굴 운동-Scielo.pdf... ✅ (12페이지, PyMuPDFLoader)
  로딩: 석곡및병풀복합추출물의피부색소침착방지및피부재생효과.pdf... ✅ (9페이지, PyMuPDFLoader)
  로딩: 남성 헤어커트의 작품제작.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: 진여수클린미스트를 이용한 중년여성의 안면피부개선 효과.pdf... ✅ (9페이지, PyMuPDFLoader)
  로딩: 성격 인상 정보가 얼굴 매력 판단에 미치는 영향.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: JKSC-2025-31-2-451.pdf... ✅ (12페이지, PyMuPDFLoader)
  로딩: 미용복지서비스품질이 노인 심리적행복감과 자아실현감에 미치는 영향.pdf... ✅ (10페이지, PyMuPDFLoader)
  로딩: 저분자 펩타이드 콤플렉스를 포함한 마이크로 스피큘 화장품의 피부 개선 효과.pdf... ✅ (18페이지, PyMuPDFLoader)
  로딩: 모던걸 헤어스타일의 구성요소와 사회적 여성성과의 상관성 연구 - 미용학 전공 여대생 관점으로.pdf... ✅ (6페이지, PyMuPDFLoader)
  로딩: From Basic Science to Basic Skin Care copy.pdf... ✅ (9페이지, PyMuPDFLoader)
  로딩: 호호바오일의모발보호효과연구.pdf... ✅ (21페이지, PyMuPDFLoader)
  로딩: Benefi

## 4. 벡터 임베딩 및 저장 (HuggingFace multilingual-e5-large)

**이 셀도 한 번만 실행하세요! 이미 임베딩이 있다면 5번 셀로 바로 가세요.**

**HuggingFace 다국어 모델 사용 - 무료!** ⭐
**한글 쿼리로 영어 논문도 검색 가능!**

In [4]:
print("=" * 50)
print("HuggingFace multilingual-e5-large 임베딩 시작...")
print(f"청크 개수: {len(chunked_documents)}개")
print("⭐ HuggingFace 다국어 모델 사용 - 무료!")
print("⭐ 한글 쿼리로 영어 논문도 검색 가능!")
print("=" * 50)

embedding_start = time.time()


# HuggingFace multilingual-e5-large 임베딩 모델 사용
# 다국어 지원으로 한글 쿼리로 영어 문서 검색 가능
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large",
    model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
    encode_kwargs={'normalize_embeddings': True}
)

print("✅ 임베딩 모델 로드 완료")
print("📥 벡터 임베딩 생성 중... (시간이 다소 걸릴 수 있습니다)")

vectorstore = Chroma.from_documents(
    documents=chunked_documents,
    embedding=embeddings,
    persist_directory="./chroma_db_tmp"
)

embedding_end = time.time()
print(f"✅ HuggingFace multilingual-e5-large 임베딩 완료: {embedding_end - embedding_start:.2f}초")
print("=" * 50)

HuggingFace multilingual-e5-large 임베딩 시작...
청크 개수: 7290개
⭐ HuggingFace 다국어 모델 사용 - 무료!
⭐ 한글 쿼리로 영어 논문도 검색 가능!
✅ 임베딩 모델 로드 완료
📥 벡터 임베딩 생성 중... (시간이 다소 걸릴 수 있습니다)
✅ HuggingFace multilingual-e5-large 임베딩 완료: 131.43초


## 5. 기존 벡터스토어 로드 (HuggingFace multilingual-e5-large)

**임베딩을 다시 하지 않고 기존 것을 사용하려면 이 셀을 실행하세요!**

**HuggingFace 다국어 모델 - 무료!** ⭐

In [5]:
if os.path.exists("./chroma_db_e5"):
    print("기존 HuggingFace multilingual-e5-large 벡터스토어 로드 중...")
    
    embeddings = HuggingFaceEmbeddings(
        model_name="intfloat/multilingual-e5-large",
        model_kwargs={'device': 'cuda'},  # GPU 사용 (없으면 'cpu'로 변경)
        encode_kwargs={'normalize_embeddings': True}
    )
    
    vectorstore = Chroma(
        persist_directory="./chroma_db_tmp",
        embedding_function=embeddings
    )
    print("✅ 벡터스토어 로드 완료")
else:
    print("❌ 벡터스토어가 없습니다. 4번 셀을 먼저 실행하세요.")

❌ 벡터스토어가 없습니다. 4번 셀을 먼저 실행하세요.


## 6. 이미지 분석 및 RAG 체인 설정

**이미지 분석 → 쿼리 재구성 → RAG 검색 방식**
- **이미지 분석**: OpenAI GPT-4o-mini Vision으로 이미지 상태 분석
- **쿼리 재구성**: 이미지 분석 결과 + 사용자 질문을 합쳐서 검색용 쿼리 생성
- **임베딩**: HuggingFace multilingual-e5-large (무료, GPU/CPU, 다국어 지원)
- **최종 답변**: OpenAI GPT-4o-mini (텍스트만)

In [6]:
# Retriever 설정
from langchain_community.retrievers import BM25Retriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

bm25_retriever = BM25Retriever.from_documents(chunked_documents)
bm25_retriever.k = 5


# OpenAI GPT-4o-mini 모델 설정
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7
)

# Vision 모델 설정 (이미지 분석용)
vision_llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)


# --- 이미지 분석 함수 (새로 추가) ---
def analyze_image(image_base64, user_query):
    """이미지를 분석하여 상태를 설명하는 텍스트 반환"""
    
    analysis_prompt = f"""당신은 전문 이미지 분석가입니다. 제공된 이미지를 분석하여 다음 카테고리별로 관찰 사항을 상세히 작성하세요.

[사용자 질문]: {user_query}

아래 카테고리별로 객관적이고 구체적으로 관찰된 사항을 작성하세요:

1. **헤어 상태**: 
   - 헤어 스타일, 길이, 볼륨, 질감
   - 색상, 광택, 손상 여부
   - 두피 상태 (보이는 경우)
   - 전반적인 헤어 컨디션

2. **피부 상태**:
   - 피부 톤, 색상
   - 수분감, 광택
   - 결, 텍스처
   - 특이사항 (잡티, 붉은기 등)

3. **얼굴 윤곽**:
   - 얼굴형
   - 눈, 코, 입의 비율 및 특징
   - 턱선, 볼살
   - 전반적인 균형감

**출력 형식**: 각 카테고리별로 3-5문장으로 관찰된 내용을 간결하고 명확하게 작성하세요.
**주의**: 부정적 표현을 피하고, 객관적 관찰에 집중하세요."""

    message = HumanMessage(
        content=[
            {"type": "text", "text": analysis_prompt},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}
            }
        ]
    )
    
    response = vision_llm.invoke([message])
    return response.content


def create_enhanced_query(user_query, image_analysis=None):
    """사용자 질문과 이미지 분석 결과를 합쳐서 검색용 쿼리 생성"""
    
    if image_analysis:
        # 이미지 분석 결과가 있는 경우
        enhanced_query = f"""[사용자 질문]
{user_query}

[이미지 분석 결과]
{image_analysis}

위 정보를 바탕으로 관련 논문 및 전문 정보를 검색합니다."""
        return enhanced_query
    else:
        # 이미지 분석 결과가 없는 경우 (텍스트만)
        return user_query


# --- MultiQuery (LLM 기반 쿼리 확장) ---
multi_query_prompt = PromptTemplate.from_template("""
Generate 3 different search queries that help retrieve relevant academic or scientific information 
related to the following question. 
Write each query on a new line.

Question: {question}
""")

generate_queries = (
    multi_query_prompt
    | ChatOpenAI(model="gpt-4o-mini", temperature=0)
    | RunnableLambda(lambda x: x.content.strip().split("\n"))
)

# --- 각 Retriever의 유사도 점수 가져오기 ---
def get_dense_retriever_with_scores(query, k=5):
    """Dense retriever (벡터 유사도)의 문서와 점수를 반환"""
    docs_and_scores = vectorstore.similarity_search_with_score(query, k=k)
    
    results = []
    for doc, score in docs_and_scores:
        results.append({
            "document": doc,
            "score": score,
            "retriever_type": "dense (vector similarity)"
        })
    return results

def get_bm25_retriever_with_scores(query, k=5):
    """BM25 retriever (키워드 기반)의 문서와 점수를 반환"""
    from rank_bm25 import BM25Okapi
    
    tokenized_corpus = [doc.page_content.split() for doc in chunked_documents]
    bm25 = BM25Okapi(tokenized_corpus)
    
    tokenized_query = query.split()
    doc_scores = bm25.get_scores(tokenized_query)
    
    top_indices = sorted(range(len(doc_scores)), key=lambda i: doc_scores[i], reverse=True)[:k]
    
    results = []
    for idx in top_indices:
        results.append({
            "document": chunked_documents[idx],
            "score": doc_scores[idx],
            "retriever_type": "sparse (BM25)"
        })
    return results

def get_all_retrievers_with_scores(query, k=5):
    """모든 retriever의 결과와 점수를 함께 반환"""
    dense_results = get_dense_retriever_with_scores(query, k)
    sparse_results = get_bm25_retriever_with_scores(query, k)
    
    print(f"\n{'='*80}")
    print(f"Query: {query}")
    print(f"{'='*80}\n")
    
    print(f"📊 Dense Retriever (Vector Similarity) - Top {k} Results:")
    print("-" * 80)
    for i, result in enumerate(dense_results, 1):
        print(f"\n[{i}] Score: {result['score']:.4f}")
        print(f"Content Preview: {result['document'].page_content[:200]}...")
        print(f"Metadata: {result['document'].metadata}")
    
    print(f"\n\n📊 Sparse Retriever (BM25) - Top {k} Results:")
    print("-" * 80)
    for i, result in enumerate(sparse_results, 1):
        print(f"\n[{i}] Score: {result['score']:.4f}")
        print(f"Content Preview: {result['document'].page_content[:200]}...")
        print(f"Metadata: {result['document'].metadata}")
    
    return {
        "dense": dense_results,
        "sparse": sparse_results
    }

# --- RRF(Reciprocal Rank Fusion) with Scores ---
def rrf_merge_with_scores(dense_docs, sparse_docs, k=60):
    """RRF를 사용하여 문서를 병합하고 최종 점수도 반환"""
    scores = {}

    def add_results(docs, source_type):
        for rank, doc in enumerate(docs):
            doc_id = doc.metadata.get("id", id(doc))
            score = 1 / (k + rank + 1)
            if doc_id not in scores:
                scores[doc_id] = {
                    "doc": doc, 
                    "rrf_score": 0,
                    "dense_rank": None,
                    "sparse_rank": None
                }
            scores[doc_id]["rrf_score"] += score
            if source_type == "dense":
                scores[doc_id]["dense_rank"] = rank + 1
            else:
                scores[doc_id]["sparse_rank"] = rank + 1

    add_results(dense_docs, "dense")
    add_results(sparse_docs, "sparse")

    merged = sorted(scores.values(), key=lambda x: x["rrf_score"], reverse=True)
    
    return merged

# --- RRF 결과 출력 함수 ---
def print_rrf_results(merged_results, top_k=7):
    """RRF 병합 결과를 보기 좋게 출력"""
    print(f"\n\n📊 RRF (Reciprocal Rank Fusion) - Top {top_k} Merged Results:")
    print("=" * 80)
    
    for i, item in enumerate(merged_results[:top_k], 1):
        print(f"\n[{i}] RRF Score: {item['rrf_score']:.4f}")
        print(f"Dense Rank: {item['dense_rank'] if item['dense_rank'] else 'N/A'}")
        print(f"Sparse Rank: {item['sparse_rank'] if item['sparse_rank'] else 'N/A'}")
        print(f"Content Preview: {item['doc'].page_content[:200]}...")
        print(f"Metadata: {item['doc'].metadata}")
        print("-" * 80)


# --- RRF(Reciprocal Rank Fusion) ---
def rrf_merge(dense_docs, sparse_docs, k=60):
    scores = {}

    def add_results(docs):
        for rank, doc in enumerate(docs):
            doc_id = doc.metadata.get("id", id(doc))
            score = 1 / (k + rank + 1)
            if doc_id not in scores:
                scores[doc_id] = {"doc": doc, "score": 0}
            scores[doc_id]["score"] += score

    add_results(dense_docs)
    add_results(sparse_docs)

    merged = sorted(scores.values(), key=lambda x: x["score"], reverse=True)
    return [item["doc"] for item in merged]


# --- MultiQuery + Hybrid Retrieval ---
def hybrid_multi_retrieve(queries):
    dense_all = []
    sparse_all = []

    for q in queries:
        dense_all.extend(retriever.invoke(q))
        sparse_all.extend(bm25_retriever.invoke(q))

    merged = rrf_merge(dense_all, sparse_all)
    return merged[:10]  # 상위 10개만 반환


hybrid_multi_retriever = (
    generate_queries
    | RunnableLambda(hybrid_multi_retrieve)
)


# LCEL 방식으로 체인 구성
def deduplicate_docs(docs):
    """중복 문서 제거"""
    seen = set()
    unique_docs = []
    for doc in docs:
        content_hash = hash(doc.page_content)
        if content_hash not in seen:
            seen.add(content_hash)
            unique_docs.append(doc)
    return unique_docs[:7]  # 최종적으로 7개만 반환

def format_docs(docs):
    """중복 제거 후 포맷팅"""
    unique_docs = deduplicate_docs(docs)
    return "\n\n".join(doc.page_content for doc in unique_docs)

def create_final_prompt(inputs):
    """최종 프롬프트 생성 (텍스트만, 이미지 제거)"""
    context = inputs["context"]
    question = inputs["question"]

    prompt_text = f"""당신은 다음 3명의 전문가가 통합된 고급 뷰티 코칭 AI입니다.
- 1. 헤어 스타일리스트: 헤어 형태, 볼륨, 질감, 스타일 방향성, 머릿결 관리 등 분석
- 2. 피부 관리 전문가: 피부 톤, 수분·유분 상태, 결, 잡티, 광택 등 관찰
- 3. 얼굴 윤곽 디자이너: 눈·코·입 비율, 턱선, 볼살, 이마 곡선, 대칭, 입체감 등 분석

[분석 방식 지침]
1. 사용자가 제공한 성별, 직업, 상황 설명을 기반으로 한다.
2. 관찰은 반드시 시각적·감각적 요소 중 최소 3가지 이상을 활용하고 디테일을 풍부하게 묘사하여, 독자가 장면을 그릴 수 있게 작성한다.
3. 부정적인 표현을 피하고 개선 가능성을 중심으로 서술한다.
4. 개선 팁은 오늘 실천 가능한 구체적 행동이나 셀프 케어 방법만 포함해야 하며, 추상적 조언은 금지된다.
5. 따뜻하고 격려적인 어조를 유지하며, 객관적이되 전문적으로 작성한다.
6. 전문 용어를 적절히 사용하되 이해하기 쉬운 표현으로 설명한다.

[논문 인용 원칙] ⭐ 중요 ⭐
- 참고 문서에 관련 논문 내용이 있으면 **반드시 직접 인용**하세요.
- 인용 시 따옴표("")를 사용하고, 구체적인 연구 결과, 수치, 전문 용어를 포함하세요.
- status와 improvement_tips 모두에서 논문 내용을 적극적으로 인용하여 전문성을 높이세요.

[분석 카테고리]
1. 헤어(Hair)
2. 피부(Skin)
3. 윤곽(Contour)

[참고 문서]
{context}

[사용자 질문 및 이미지 분석]
{question}

[Output JSON Example]
{{{{
  "Appearance_Coaching": {{{{
    "Hair": {{{{
      "status": "헤어의 현재 상태에 대한 간결한 요약 설명. 만약 사용자가 헤어 관련 질문(염색, 스타일 등)을 했다면, 여기에 구체적인 추천을 포함하여 작성. 참고 문서의 논문 내용이 있다면 따옴표를 사용하여 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1",
        "오늘 실천 가능한 구체적인 행동 2"
      ]
    }}}},
    "Skin": {{{{
      "status": "피부의 현재 상태에 대한 간결한 요약 설명. 만약 사용자가 피부/메이크업 관련 질문을 했다면, 여기에 구체적인 추천을 포함하여 작성. 논문 내용 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1",
        "오늘 실천 가능한 구체적인 행동 2"
      ]
    }}}},
    "Contour": {{{{
      "status": "윤곽의 현재 상태에 대한 간결한 요약 설명. 논문 내용 직접 인용",
      "improvement_tips": [
        "오늘 실천 가능한 구체적인 행동 1",
        "오늘 실천 가능한 구체적인 행동 2"
      ]
    }}}}
  }}}}
}}}}

**중요 규칙:**
1. 답변은 반드시 한글로만 작성해야 합니다. 영어는 절대 사용하지 마세요.
2. 출력은 반드시 위 JSON 형식으로 하며, 개선 팁은 반드시 오늘 실천 가능한 구체적 행동으로 작성하세요.
3. 사용자 질문이 특정 주제(염색, 헤어스타일, 메이크업 등)에 대한 것이면, 해당 카테고리의 status에 질문에 대한 구체적인 답변을 우선적으로 포함하세요.
4. status는 시각적·감각적 요소를 3가지 이상 포함하되, 질문에 대한 답변을 먼저 작성하고 현재 상태 분석을 추가하세요.
5. **참고 문서에 논문 내용이 있으면 반드시 따옴표("")를 사용하여 직접 인용하고, 연구 결과나 전문 지식을 적극 활용하세요.**

답변:"""

    return [HumanMessage(content=prompt_text)]

qa_chain = (
    {
        "context": lambda x: format_docs(hybrid_multi_retriever.invoke({"question": x["question"]})),
        "question": lambda x: x["question"]
    }
    | RunnableLambda(create_final_prompt)
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인 설정 완료")
print("✅ 이미지 분석 함수 추가:")
print("   - analyze_image(image_base64, user_query): 이미지를 분석하여 상태 설명 텍스트 반환")
print("   - create_enhanced_query(user_query, image_analysis): 쿼리 + 이미지 분석 결과 병합")
print("✅ 각 retriever의 유사도 점수 조회 함수:")
print("   - get_dense_retriever_with_scores(query, k=5)")
print("   - get_bm25_retriever_with_scores(query, k=5)")
print("   - get_all_retrievers_with_scores(query, k=5)")
print("   - rrf_merge_with_scores(dense_docs, sparse_docs, k=60)")
print("   - print_rrf_results(merged_results, top_k=7)")

✅ RAG 체인 설정 완료
✅ 이미지 분석 함수 추가:
   - analyze_image(image_base64, user_query): 이미지를 분석하여 상태 설명 텍스트 반환
   - create_enhanced_query(user_query, image_analysis): 쿼리 + 이미지 분석 결과 병합
✅ 각 retriever의 유사도 점수 조회 함수:
   - get_dense_retriever_with_scores(query, k=5)
   - get_bm25_retriever_with_scores(query, k=5)
   - get_all_retrievers_with_scores(query, k=5)
   - rrf_merge_with_scores(dense_docs, sparse_docs, k=60)
   - print_rrf_results(merged_results, top_k=7)


In [7]:
test_query = "I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem."
hybrid_multi_retriever.invoke(test_query)

#get_dense_retriever_with_scores(test_query, k = 5)
# get_bm25_retriever_with_scores(test_query, k=5)
get_all_retrievers_with_scores(test_query, k=5)


Query: I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem.

📊 Dense Retriever (Vector Similarity) - Top 5 Results:
--------------------------------------------------------------------------------

[1] Score: 0.2802
Content Preview: 1040-50.
7.
Lauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.
Prevalence and odds of depressive and anxiety disorders and symptoms in
children and adults with alopecia areata:...
Metadata: {'creator': 'Adobe InDesign 19.5 (Macintosh)', 'creationDate': "D:20251009095737+03'00'", 'moddate': '2025-10-09T09:57:37+03:00', 'page': 4, 'modDate': "D:20251009095737+03'00'", 'author': '', 'title': '', 'file_path': 'data/AA–불안·우울 연관 메타분석.pdf', 'format': 'PDF 1.4', 'creationdate': '2025-10-09T09:57:37+03:00', 'producer': 'Adobe PDF Library 17.0', 'trapped': '', 'total_pages': 5, 'subject': '', 'source': 'data/AA–불안·우울 연관 메타분석.pdf', 'keywords': ''}

[2] Score: 0.2802
Con

{'dense': [{'document': Document(metadata={'creator': 'Adobe InDesign 19.5 (Macintosh)', 'creationDate': "D:20251009095737+03'00'", 'moddate': '2025-10-09T09:57:37+03:00', 'page': 4, 'modDate': "D:20251009095737+03'00'", 'author': '', 'title': '', 'file_path': 'data/AA–불안·우울 연관 메타분석.pdf', 'format': 'PDF 1.4', 'creationdate': '2025-10-09T09:57:37+03:00', 'producer': 'Adobe PDF Library 17.0', 'trapped': '', 'total_pages': 5, 'subject': '', 'source': 'data/AA–불안·우울 연관 메타분석.pdf', 'keywords': ''}, page_content='1040-50.\n7.\nLauron S, Plasse C, Vaysset M, Pereira B, D’Incan M, Rondepierre F, et al.\nPrevalence and odds of depressive and anxiety disorders and symptoms in\nchildren and adults with alopecia areata: a systematic review and metaanalysis. JAMA Dermatol. 2023; 159(3): 281-8.\n8.\nKurd SK, Troxel AB, Crits-Christoph P, Gelfand JM. The risk of depression,\nanxiety, and suicidality in patients with psoriasis: a population-based cohort\nstudy. Arch Dermatol. 2010; 146(8): 891-5.\n9.\n

## 7. 질의응답 실행 (텍스트만)

**이미지 없이 텍스트만 사용하는 경우**

In [42]:
# 질문을 여기서 수정하세요
query = "I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem."

print("=" * 50)
print(f"질문: {query}")
print("=" * 50)
print("\nRAG 쿼리 시작... (OpenAI API 호출 중)")
invoke_start = time.time()

result = qa_chain.invoke({"question": query})

invoke_end = time.time()
print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
print("=" * 50)

print("\n📝 답변:")
print(result)

질문: I have alopecia areata and it’s affecting my confidence. Please suggest ways to improve both my hair condition and self-esteem.

RAG 쿼리 시작... (OpenAI API 호출 중)


KeyboardInterrupt: 

## 7. 질의응답 실행 (이미지 포함)

**이미지 분석 → 쿼리 재구성 → RAG 검색 → 답변 생성**

1. 이미지를 OpenAI Vision API로 먼저 분석
2. 이미지 분석 결과 + 사용자 질문을 합쳐서 검색 쿼리 생성
3. RAG 검색 수행
4. 최종 답변 생성 (텍스트만)

In [8]:
# 이미지 경로와 질문을 수정하세요
image_path = "1.jpg"
query = "지금 머리가 이모냥인데 어떻게해야해?"

if Path(image_path).exists():
    print("=" * 50)
    print(f"이미지: {image_path}")
    print(f"질문: {query}")
    print("=" * 50)
    
    # 1단계: 이미지 인코딩
    print("\n[1단계] 이미지 인코딩 중...")
    image_base64 = encode_image(image_path)
    print("✅ 이미지 인코딩 완료")
    
    # 2단계: 이미지 분석 (Vision API 호출)
    print("\n[2단계] 이미지 분석 중... (OpenAI Vision API 호출)")
    analysis_start = time.time()
    image_analysis = analyze_image(image_base64, query)
    analysis_end = time.time()
    print(f"✅ 이미지 분석 완료: {analysis_end - analysis_start:.2f}초")
    print("\n📊 이미지 분석 결과:")
    print("-" * 50)
    print(image_analysis)
    print("-" * 50)
    
    # 3단계: 강화된 쿼리 생성
    print("\n[3단계] 검색용 쿼리 생성 중...")
    enhanced_query = create_enhanced_query(query, image_analysis)
    print("✅ 쿼리 생성 완료")
    print("\n📝 검색용 쿼리 (미리보기):")
    print(enhanced_query[:300] + "...")
    
    # 4단계: RAG 검색 및 답변 생성
    print("\n[4단계] RAG 검색 및 답변 생성 중... (OpenAI API 호출)")
    invoke_start = time.time()
    
    result = qa_chain.invoke({"question": enhanced_query})
    
    invoke_end = time.time()
    print(f"✅ RAG 쿼리 완료: {invoke_end - invoke_start:.2f}초")
    print("=" * 50)
    
    print("\n📝 최종 답변:")
    print(result)
    
    # 전체 소요 시간
    total_time = (analysis_end - analysis_start) + (invoke_end - invoke_start)
    print("\n" + "=" * 50)
    print(f"⏱️ 전체 소요 시간: {total_time:.2f}초")
    print(f"  - 이미지 분석: {analysis_end - analysis_start:.2f}초")
    print(f"  - RAG 검색 및 답변: {invoke_end - invoke_start:.2f}초")
    print("=" * 50)
else:
    print(f"❌ 이미지 파일을 찾을 수 없습니다: {image_path}")

이미지: 1.jpg
질문: 지금 머리가 이모냥인데 어떻게해야해?

[1단계] 이미지 인코딩 중...
✅ 이미지 인코딩 완료

[2단계] 이미지 분석 중... (OpenAI Vision API 호출)
✅ 이미지 분석 완료: 10.89초

📊 이미지 분석 결과:
--------------------------------------------------
1. **헤어 상태**: 
   - 헤어 스타일은 자연스럽게 떨어지는 긴 머리로 보이며, 길이는 중간 정도입니다. 
   - 볼륨은 적당하며, 질감은 부드러운 느낌을 줍니다. 
   - 색상은 어두운 톤으로, 광택이 약간 있으며, 손상된 부분은 보이지 않습니다. 
   - 두피는 보이지 않지만, 전반적으로 건강한 헤어 컨디션으로 판단됩니다.

2. **피부 상태**: 
   - 피부 톤은 밝고 고른 색상으로 보입니다. 
   - 수분감이 느껴지며, 약간의 광택이 있습니다. 
   - 결은 매끄럽고, 텍스처는 부드러운 느낌을 줍니다. 
   - 특이사항으로는 약간의 붉은기가 보이지만, 전반적으로 깨끗한 상태입니다.

3. **얼굴 윤곽**: 
   - 얼굴형은 타원형으로, 균형 잡힌 인상을 줍니다. 
   - 눈, 코, 입의 비율이 조화롭고, 각 부위의 특징이 잘 드러납니다. 
   - 턱선은 부드럽고, 볼살은 적당하여 전체적인 균형감이 좋습니다. 
   - 전반적으로 얼굴 윤곽이 매력적으로 보입니다.
--------------------------------------------------

[3단계] 검색용 쿼리 생성 중...
✅ 쿼리 생성 완료

📝 검색용 쿼리 (미리보기):
[사용자 질문]
지금 머리가 이모냥인데 어떻게해야해?

[이미지 분석 결과]
1. **헤어 상태**: 
   - 헤어 스타일은 자연스럽게 떨어지는 긴 머리로 보이며, 길이는 중간 정도입니다. 
   - 볼륨은 적당하며, 질감은 부드러운 느낌을 줍니다. 
   - 색상은 어두운 톤으로, 광택이 약간 있으며, 손상된 부분은 보이지 않습니다. 


In [9]:
# 마지막 질문에 대한 참고 문서 확인
retrieved_docs = hybrid_multi_retriever.invoke(query)

print("📚 참고 문서:")
print("=" * 50)
for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[문서 {i}]")
    print(doc.page_content)
    print("-" * 50)

📚 참고 문서:

[문서 1]
속상하거나 우울할 때 헤어스타일 바꾸는 것이감정 치유에 영향을
끼친다고 생각한다.
.245
.755
.630
나는 스타일 변화가 만족스러우면 기분전환이 되고 스트레스가
풀리는것 같다.
.178
.752
.597
의욕이 없고 짜증이 많아질 때 헤어스타일을 바꾸면 완화될거라
생각한다.
.242
.661
.496
고유치
2.899
2.560
설명변량(%)
28.989
25.601
누적변량(%)
28.989
54.590
신뢰도계수(Cronbach'sα)
.801
.780
<Table 2> Validity and Reliability of Cmotional Healing
--------------------------------------------------

[문서 2]
어스타일 관리는 자신의 개성을 표현하며 외적
이미지로 변화를 줄수있는 중요한 수단으로써
의자기표현방법중 하나로 보여지고 있다.
한효주의 여성 헤어스타일 결정 요인과 관련된 선행연구에서는 20~40대 여성의 헤어스타
일 인식도와 선호도 연구8)에서는 여성들의 헤
어이미지 선호도와 헤어스타일 연출성향의 선
호도를 구체적으로 알아보는 연구를 하였고, 김
미선의 헤어스타일 결정에 영향을 미치는 개인적 요인과 헤어태도에 관한 연구9)에서는 헤어
스타일 변화의도에 영향을 미친후 행동으로 어떻게 이어지는지에 대해 분석하였으며, 이영애
의 여대생의 일반적 특성이 헤어스타일 결정에
미치는 영향에 관한 연구10)에서는 헤어스타일
에대한 정보를 제공하고 헤어스타일 결정에 다양한 욕구를 충족시키는지에 대해 연구를 하였
으며, 강수희의 헤어스타일별 시술주기와 헤어
스타일 변화에 영향을 주는 요인분석에 관한연
구11)에서는 미용실 손님들의 헤어스타일 관찰을 통하여 헤어스타일 변화 주기와 관련 요인을
연구하였다.
하지만 MZ세대 여성을 중심으로 얼굴이미지와 헤어스타일의 분석에 대한 연구는 찾아보
기 어렵다. 따라서 본 연구는 디지털문화에 익
숙하며, 유행에 민감하게 반응하고, SNS를 통한핵심소

In [ ]:
# 예시 3: RRF 병합 결과와 점수 확인
test_query = "원형탈모와 자존감의 관계"

print("\n" + "=" * 80)
print(f"테스트 쿼리: {test_query}")
print("=" * 80)

# 1. Dense와 Sparse retriever로 문서 가져오기
dense_docs = [result['document'] for result in get_dense_retriever_with_scores(test_query, k=5)]
sparse_docs = [result['document'] for result in get_bm25_retriever_with_scores(test_query, k=5)]

# 2. RRF로 병합하고 점수 확인
merged_results = rrf_merge_with_scores(dense_docs, sparse_docs, k=60)

# 3. 결과 출력
print_rrf_results(merged_results, top_k=7)

In [ ]:
# 예시 2: get_all_retrievers_with_scores 함수로 한 번에 확인
test_query = "탈모 치료 방법"

results = get_all_retrievers_with_scores(test_query, k=5)

In [ ]:
# 예시 1: 각 retriever의 점수를 개별적으로 확인
test_query = "헤어 염색이 모발에 미치는 영향"

print("=" * 80)
print(f"테스트 쿼리: {test_query}")
print("=" * 80)

# Dense Retriever (벡터 유사도) 결과
print("\n📊 1. Dense Retriever (Vector Similarity) 결과:")
print("-" * 80)
dense_results = get_dense_retriever_with_scores(test_query, k=5)
for i, result in enumerate(dense_results, 1):
    print(f"\n[{i}] Score: {result['score']:.4f}")
    print(f"Content Preview: {result['document'].page_content[:150]}...")
    print(f"Source: {result['document'].metadata.get('source', 'N/A')}")

# BM25 Retriever (키워드 기반) 결과
print("\n\n📊 2. BM25 Retriever (Keyword-based) 결과:")
print("-" * 80)
sparse_results = get_bm25_retriever_with_scores(test_query, k=5)
for i, result in enumerate(sparse_results, 1):
    print(f"\n[{i}] Score: {result['score']:.4f}")
    print(f"Content Preview: {result['document'].page_content[:150]}...")
    print(f"Source: {result['document'].metadata.get('source', 'N/A')}")

## 9. 각 Retriever의 유사도 점수 확인

각 retriever의 점수를 확인하고 RRF로 병합된 결과를 비교합니다.